In [68]:
#Necessary imports

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import pandas as pd
import dlib
import seaborn as sns


from scipy.signal import butter, filtfilt, lfilter
from scipy.fft import fft, fftfreq
from datetime import datetime
from scipy.signal import find_peaks


In [69]:
#Constants
low_cutoff = 0.75
high_cutoff = 2.5
fps=0
fs = 2 * high_cutoff

#Paths
dlib_path = 'dlib_files/shape_predictor_68_face_landmarks.dat'
# Load face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dlib_path)


#pulses values
pulses_forehead=[]
pulses_leftc=[]
pulses_rightc=[]
# pulses_fullf=[]

# Create a new DataFrame with new columns data


In [70]:
#Utility functions

# def setValues():
#     #Constants
#     low_cutoff = 0.75
#     high_cutoff = 2.5
#     fs = 2 * high_cutoff    
#     #pulses values
#     # pulses_fullf=[]


def calculate_pulse_signal(roi,pulses):
    
    B, G, R = cv2.split(roi)
    
    #Normalizing R,G,B channels
    
    Rn = R / np.mean(R)
    Gn = G / np.mean(G)
    Bn = B / np.mean(B)    
    
    #CHROM Signals
    
    Xs = 3*Rn - 2*Gn
    Ys = 1.5*Rn + Gn - 1.5*Bn
    
    Rf = apply_bandpass_filter(Rn)
    Gf = apply_bandpass_filter(Gn)
    Bf = apply_bandpass_filter(Bn)
    Xf = apply_bandpass_filter(Xs)
    Yf = apply_bandpass_filter(Ys)

    alpha = np.std(Xf) / np.std(Yf)

    Signal = 3*(1-(alpha/2))*Rf - 2*(1 + (alpha/2))*Gf + ((3*alpha)/2)*Bf
    
    pulses.append(np.mean(Signal))  

def apply_bandpass_filter(channel, order=5):
    b, a = butter_bandpass(order)
    y = filtfilt(b, a, channel, axis=0,padlen=0)
    return y

def butter_bandpass(order=5):
    nyquist = 0.5 * fs  # Nyquist frequency
    lowcut_norm = low_cutoff / nyquist 
    highcut_norm = high_cutoff / nyquist
    b, a = butter(order, [lowcut_norm, highcut_norm-0.1], btype='band')
    return b, a

def find_peak(pulses):
    peaks, pro = find_peaks(pulses , height=np.mean(pulses))
    # Output results
    # print("Indices of peaks:", peaks)
    # print("pro: ",pro['peak_heights'])
    return peaks, pro['peak_heights']
    

In [71]:
# Open video file

def calculateResult(file_path):
    data = {
    'peakscf' : np.nan,
    'peakscl' : np.nan,
    'peakscr' : np.nan,
    'maxif': np.nan,
    'maxil':np.nan,
    'maxir':np.nan,
    'avgif':np.nan,
    'avgil':np.nan,
    'avgir':np.nan,
    'commonpeaksc': np.nan,
    'facecount': np.nan,
    'error':0
    }
    
    cap = cv2.VideoCapture(file_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    global pulses_forehead,pulses_leftc,pulses_rightc
    
    pulses_forehead=[]
    pulses_leftc=[]
    pulses_rightc=[]
    
    # Process each frame in the video
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
    
        # Convert frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = detector(gray)
        
        if(len(faces) > 1):
            data['facecount'] = len(faces)
            print('more than one face')
            return data
            break
    
        # Process each detected face
        for face in faces:
            # Detect facial landmarks
            landmarks = predictor(gray, face)
            
            # Identify forehead region based on landmarks
            forehead_r1=[0,0,0,0] #x1,x2,y1,y2
            
            forehead_r1[0] = landmarks.part(21).x # Left eyebrow-x
            forehead_r1[1] = landmarks.part(22).x  # Right eyebrow-x
            forehead_r1_height=int((forehead_r1[1] - forehead_r1[0]))
            forehead_r1[2] = landmarks.part(21).y - forehead_r1_height # Left eyebrow-y
            forehead_r1[3] = landmarks.part(22).y # Right eyebrow-y
            
            # Identify left cheek region-1 based on landmarks
            leftc_r1=[0,0,0,0] #x1,x2,y1,y2
            
            leftc_r1[0]=landmarks.part(41).x
            leftc_r1[1]=landmarks.part(39).x
            leftc_r1[2]=landmarks.part(28).y
            leftc_r1[3]=landmarks.part(30).y
    
            # Identify right cheek region-1 based on landmarks
            rightc_r1=[0,0,0,0] #x1,x2,y1,y2
            
            rightc_r1[0]=landmarks.part(42).x
            rightc_r1[1]=landmarks.part(46).x
            rightc_r1[2]=landmarks.part(28).y
            rightc_r1[3]=landmarks.part(30).y
            
            try:
                calculate_pulse_signal(frame[forehead_r1[2]:forehead_r1[3],forehead_r1[0]:forehead_r1[1]],pulses_forehead)
                calculate_pulse_signal(frame[leftc_r1[2]:leftc_r1[3],leftc_r1[0]:leftc_r1[1]],pulses_leftc)
                calculate_pulse_signal(frame[rightc_r1[2]:rightc_r1[3],rightc_r1[0]:rightc_r1[1]],pulses_rightc)   
            except Exception as e:
                print(e,file_path)
                data['error']=1
                return data
    
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            # handle if q pressed
            break
    # Release video capture
    cap.release()
    # print(pulses_forehead)
    peaksf, hf=find_peak(pulses_forehead)
    peaksl, hl=find_peak(pulses_leftc)
    peaksr, hr=find_peak(pulses_rightc)

    common_peaks = set(peaksf).intersection(peaksl, peaksr)
    # print('common i:',common_peaks)
    
    data = {
    'peakscf' : len(peaksf),
    'peakscl' : len(peaksl),
    'peakscr' : len(peaksr),
    'maxif': np.max(hf),
    'maxil':np.max(hl),
    'maxir':np.max(hr),
    'avgif':np.mean(pulses_forehead),
    'avgil':np.mean(pulses_leftc),
    'avgir':np.mean(pulses_rightc),
    'commonpeaksc' :len(common_peaks),
    'facecount': 1,
    'error':0
    }    
    return data


In [72]:
excel_file = 'celebDF_DS.xlsx'  # Replace with the path to your Excel file
output_file='peaks.xlsx'
# video_path = 'videos/real/jenny.mp4'
results=[]

# Read the existing Excel file    
existing_df = pd.read_excel(excel_file)
# print(calculateResult(video_path))
count=0
for filepath in existing_df['filepath']:
        print(f"processing: {filepath}")
        result = calculateResult(filepath)
        results.append(result)
# updating Results DataSet
# print(results)
try:
    results_df = pd.DataFrame(results)
    results_df.to_excel(output_file,index=False)
except Exception as e:
    print(f"An error occurred: {e}")

Indices of peaks: [  1   4   8  24  29  38  47  52  55  59  64  67  69  72  75  79  81  84
  87  91  94  97  99 114 118 122 126 128 138 172 175 177 216 219 222 224
 240 256 265 268 278 290 292 312 315 321 324 328 332 340 344 351 354 357
 359 367 371]
pro:  [1.84794367e-05 1.31708805e-05 1.63462777e-05 1.02576574e-05
 2.57445283e-06 8.53556407e-06 1.15660748e-05 2.32742751e-05
 1.00393666e-04 5.05026329e-05 1.59279040e-04 1.38297162e-04
 8.86394354e-06 1.11876914e-04 3.69779512e-05 8.98818471e-05
 4.90698723e-05 5.76493140e-05 1.63041086e-05 1.27262027e-04
 1.32390593e-04 8.24152130e-06 6.90879795e-06 3.89395091e-06
 9.54225534e-06 8.35967953e-06 7.03258549e-06 1.71663317e-05
 8.37344340e-06 3.40041433e-06 1.04194846e-05 3.75261524e-06
 7.56498474e-06 8.89497877e-06 6.60649638e-06 7.88687953e-06
 3.47823511e-06 4.75411236e-06 4.16522259e-06 3.72486786e-06
 5.42074201e-06 8.39880149e-06 5.44197248e-06 2.12464033e-05
 2.43878517e-05 1.37037511e-05 2.53311296e-05 8.84157194e-06
 7.21698247

Indices of peaks: [  3   6   8  10  17  22  39  42  44  48  50  54  58  60  63  67  70  75
  77  83  92  94  96 103 106 108 114 116 121 125 128 135 138 141 144 153
 170 172 175 178 183 203 232 235 238 241 245 247 255 258 262 266 279 286
 300 304 309 314 317 320 322 324 330 336 338 354 358 365 369 373]
pro:  [ 9.60844923e-07  3.26898081e-05  3.26898081e-05  3.26898081e-05
  8.28695703e-06  6.61229375e-06  3.52654008e-05  3.53333632e-05
  2.30199325e-05 -1.34060867e-06  3.60930649e-05 -4.82896305e-07
  1.97190704e-05  1.22750235e-04  9.35016790e-05  7.17424844e-05
  3.01985965e-05  3.01253015e-06  3.55245009e-06  9.55032619e-06
  2.65326451e-06 -9.46888863e-07  1.71084299e-05  1.78426530e-05
 -1.98453103e-06  1.68128447e-05 -1.30769990e-06  1.75998721e-05
  7.87111946e-06  1.04413718e-05 -9.40188770e-07  5.70053079e-06
  1.13248344e-05  1.10583012e-05  2.05972513e-05  2.01585801e-05
  7.99507139e-06  8.15742683e-06  4.91462550e-06  5.36489470e-06
  1.14316473e-05  5.73988439e-06 -1.52634